In [ ]:
import deepl
from dotenv import dotenv_values
from pathlib import Path

config = dotenv_values(".env")

auth_key = config['auth_key']
print(auth_key)

In [2]:
translator = deepl.Translator(auth_key)
usage = translator.get_usage()
print(usage.character)

0 of 500000


In [3]:
root_path = Path('el_files')
if not root_path.is_dir():
    raise ValueError("Root path is not a folder!")

In [9]:
def errorProofTranslate(ipath:Path,opath:Path):
    try:
        # Using translate_document_from_filepath() with file paths 
        translator.translate_document_from_filepath(
            ipath,
            opath,
            source_lang="EL",
            target_lang="EN-GB",
        )
        
    except deepl.DocumentTranslationException as error:
        # If an error occurs during document translation after the document was
        # already uploaded, a DocumentTranslationException is raised. The
        # document_handle property contains the document handle that may be used to
        # later retrieve the document from the server, or contact DeepL support.
        doc_id = error.document_handle.id
        doc_key = error.document_handle.key
        print(f"Error after uploading ${error}, id: ${doc_id} key: ${doc_key}")
    except deepl.DeepLException as error:
        # Errors during upload raise a DeepLException
        print(error)

In [10]:
project_file = Path(r"D:\U_targyak\ERASMUS\AI\Project_ΥΝ_2022-23_ΜέροςΒ.pdf")
out_file = project_file.parent/(project_file.stem + "_eng" + project_file.suffix)
errorProofTranslate(project_file,out_file)

In [40]:
import os
items = root_path.rglob('*')
dir_routing = {
    root_path:Path('translated')/root_path
}
os.makedirs(Path('translated')/root_path,exist_ok=True)
for I in items:
    if I.is_dir():
        #translate dir name
        folder_name = translator.translate_text(I.name,source_lang="el",target_lang="en-gb")
        dir_routing[I] = Path('translated')/I.parent / str(folder_name)
        os.makedirs(dir_routing[I],exist_ok=True)
    elif I.is_file():
        if I.suffix in ['.pdf','.doc','.ppt','.docx','.pptx']:
            if I.parent not in dir_routing:
                #the dirs should appear earlier, but catch if there is an error
                raise ValueError("Unexpected path")
            parent_path = dir_routing[I.parent]
            file_name = translator.translate_text(I.stem,source_lang="el",target_lang="en-gb")
            #print("fname: ",file_name," suffix: ",I.suffix)
            save_path = parent_path/(str(file_name)+I.suffix)
            print("Translating: ",I.name)
            errorProofTranslate(I,save_path)
#cheating on the file root renaming
(Path('translated')/root_path).rename("en_files")
os.rmdir(Path('translated'))

Translating:  Project_ΥΝ_2022-23_Μέρος-Α.pdf
Translating:  ΥΠΟΛΟΓΙΣΤΙΚΗ ΝΟΗΜΟΣΥΝΗ_ΥΛΗ.docx
Translating:  ΥΝ-Lecture1.pdf
Translating:  ΥΝ-Lecture2-perceptron.pdf
Translating:  K-fold cross-validation & Weight decay.docx
Translating:  Παρουσίαση WEKA.ppt
Bad request, message: Failed to get the document_type from the file extension (Παρουσίαση WEKA.ppt).
Translating:  Φροντιστήριο 1.pptx
Translating:  Φροντιστήριο 10.pptx
Translating:  Φροντιστήριο 2.pptx
Translating:  Φροντιστήριο 3.pptx


AttributeError: 'DocumentHandle' object has no attribute 'id'

In [22]:
status = translator.translate_document_from_filepath(test_file,'test.pdf',source_lang="el",target_lang="en-gb")